In [ ]:
MySQL Physical Cold Backup
=============================================

-- Cold backup involves physically copying database related files to backup device
   taking the database down.
-- for this the mysql database server should be clean shutdown.

files required in cold backup
==========================================

-- all the files in datadir.
-- all the files of system tablespaces.
-- all the configuration/option files.

files not required in cold backup
============================================

-- redo log files , undo log files , bin log files 
-- temp files , double write buffer files.


mysql> show variables like '%datadir%';
+---------------+----------------------+
| Variable_name | Value                |
+---------------+----------------------+
| datadir       | /var/lib/mysql/data/ |
+---------------+----------------------+
1 row in set (0.01 sec)

mysql> show variables like '%innodb_data%';
+-----------------------+------------------------------------+
| Variable_name         | Value                              |
+-----------------------+------------------------------------+
| innodb_data_file_path | ibdata1:12M;ibdata2:20M:autoextend |
| innodb_data_home_dir  | /var/lib/mysql/systemdata/         |
+-----------------------+------------------------------------+
2 rows in set (0.01 sec)


clean shutdown the database
=======================================

mysql> set global innodb_fast_shutdown = 0;
Query OK, 0 rows affected (0.00 sec)

[root@mysqlserver mysql]# systemctl stop mysqld
[root@mysqlserver mysql]# 


copy files to backup location 
=========================================

[root@mysqlserver mysql]# cd /var/lib/mysql
[root@mysqlserver mysql]# 
[root@mysqlserver mysql]# ls -rtl
total 8
drwxr-xr-x. 2 mysql mysql   29 May 19 20:38 errorlog
drwxr-xr-x. 2 mysql mysql    6 May 19 20:45 tmpdir
drwxr-xr-x. 2 mysql mysql   81 May 20 00:12 doublewrite
drwxr-xr-x. 3 mysql mysql   26 May 20 02:16 redologs
drwxr-xr-x. 2 mysql mysql   58 May 20 13:22 undolog
drwxr-xr-x. 2 mysql mysql 4096 May 20 15:09 binlog
drwxr-xr-x. 7 mysql mysql 4096 May 20 17:23 data
drwxr-xr-x. 2 mysql mysql    6 May 20 17:23 tempdata
drwxr-xr-x. 2 mysql mysql   58 May 20 17:23 systemdata
[root@mysqlserver mysql]# 

[root@mysqlserver mysql]# cp -r /var/lib/mysql/data /tmp/mysql_cold_backup/
[root@mysqlserver mysql]# 
[root@mysqlserver mysql]# cp -r /var/lib/mysql/systemdata /tmp/mysql_cold_backup/
[root@mysqlserver mysql]# 

[root@mysqlserver mysql]# cp -p /etc/mysql/my.cnf /tmp/mysql_cold_backup/
[root@mysqlserver mysql]# 
[root@mysqlserver mysql]# cp -p /var/lib/mysql-files/*.cnf /tmp/mysql_cold_backup/
[root@mysqlserver mysql]# 

[root@mysqlserver mysql]# ls -rtl /tmp/mysql_cold_backup/
total 12
-rw-r--r--. 1 root  root   425 May 19 20:47 my.cnf
-rw-r--r--. 1 mysql mysql  691 May 20 15:08 innodb.cnf
drwxr-xr-x. 7 root  root  4096 May 20 17:25 data
drwxr-xr-x. 2 root  root    58 May 20 17:25 systemdata
[root@mysqlserver mysql]# 


cold backup is completed , start the database server 
=========================================================

[root@mysqlserver mysql]# systemctl start mysqld
[root@mysqlserver mysql]# 



simulate data corruption by dropping database 
=========================================================

mysql> show databases;
+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| performance_schema |
| posdb              |
| sakila             |
| sys                |
+--------------------+
6 rows in set (0.00 sec)

mysql> drop database sakila;
Query OK, 23 rows affected (0.40 sec)

mysql> drop database posdb;
Query OK, 2 rows affected (0.01 sec)

mysql> show databases;
+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| performance_schema |
| sys                |
+--------------------+
4 rows in set (0.00 sec)


[root@mysqlserver mysql]# rm -rf /var/lib/mysql/systemdata/*
[root@mysqlserver mysql]#

[root@mysqlserver mysql]# rm -rf /var/lib/mysql/data/*
[root@mysqlserver mysql]#

[root@mysqlserver mysql]# systemctl stop mysqld
[root@mysqlserver mysql]# 
[root@mysqlserver mysql]# systemctl start mysqld
Job for mysqld.service failed because the control process exited with error code.
See "systemctl status mysqld.service" and "journalctl -xe" for details.
[root@mysqlserver mysql]# 


[root@mysqlserver mysql]# tail /var/lib/mysql/errorlog/posb_mysqld.log 
2024-05-20T12:07:57.809540Z 0 [ERROR] [MY-010119] [Server] Aborting
2024-05-20T12:07:58.623504Z 1 [ERROR] [MY-011011] [Server] Failed to find valid data directory.
2024-05-20T12:07:58.623952Z 0 [ERROR] [MY-010020] [Server] Data Dictionary initialization failed.
2024-05-20T12:07:58.623981Z 0 [ERROR] [MY-010119] [Server] Aborting
2024-05-20T12:07:58.624477Z 0 [System] [MY-010910] [Server] /usr/sbin/mysqld: Shutdown complete 

[root@mysqlserver mysql]# systemctl stop mysqld
[root@mysqlserver mysql]# 



Restore from the cold backup taken earlier 
=========================================================

remove all the files before restoring
=========================================

[root@mysqlserver mysql]# rm -rf /var/lib/mysql/tmpdir/*
[root@mysqlserver mysql]# 
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/doublewrite/*
[root@mysqlserver mysql]# 
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/redologs/*
[root@mysqlserver mysql]# 
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/undolog/*
[root@mysqlserver mysql]# 
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/tempdata/*
[root@mysqlserver mysql]# 
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/data/*
[root@mysqlserver mysql]# 
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/binlog/*
[root@mysqlserver mysql]# 
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/systemdata/*
[root@mysqlserver mysql]# 

copy the files from backup
=================================

[root@mysqlserver mysql_cold_backup]# cp -r /tmp/mysql_cold_backup/data/* /var/lib/mysql/data/
[root@mysqlserver mysql_cold_backup]# 
[root@mysqlserver mysql_cold_backup]# cp -r /tmp/mysql_cold_backup/systemdata/* /var/lib/mysql/systemdata/
[root@mysqlserver mysql_cold_backup]# 

[root@mysqlserver mysql_cold_backup]# chown -R mysql:mysql /var/lib/mysql/data/
[root@mysqlserver mysql_cold_backup]# 
[root@mysqlserver mysql_cold_backup]# chown -R mysql:mysql /var/lib/mysql/systemdata/
[root@mysqlserver mysql_cold_backup]#

[root@mysqlserver mysql]# systemctl start mysqld
Job for mysqld.service failed because the control process exited with error code.
See "systemctl status mysqld.service" and "journalctl -xe" for details.
[root@mysqlserver mysql]# 

2024-05-20T12:20:18.690519Z 1 [ERROR] [MY-013862] [InnoDB] Neither found #innodb_redo subdirectory, 
nor ib_logfile* files in /var/lib/mysql/redologs/
2024-05-20T12:20:18.690602Z 1 [ERROR] [MY-012930] [InnoDB] Plugin initialization aborted with error Generic error.
2024-05-20T12:20:19.166551Z 1 [ERROR] [MY-010334] [Server] Failed to initialize DD Storage Engine.

[root@mysqlserver mysql]# mkdir /var/lib/mysql/redologs/'#innodb_redo'
[root@mysqlserver mysql]# 
[root@mysqlserver mysql]# chown -R mysql:mysql /var/lib/mysql/redologs/
[root@mysqlserver mysql]# 

[root@mysqlserver mysql]# systemctl start mysqld
[root@mysqlserver mysql]# 

mysql> show databases;
+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| performance_schema |
| posdb              |
| sakila             |
| sys                |
+--------------------+
6 rows in set (0.01 sec)

mysql> use sakila;

mysql> show tables;
+----------------------------+
| Tables_in_sakila           |
+----------------------------+
| actor                      |
| actor_info                 |
| address                    |
| category                   |

mysql> select * from actor;
+----------+-------------+--------------+---------------------+
| actor_id | first_name  | last_name    | last_update         |
+----------+-------------+--------------+---------------------+
|        1 | PENELOPE    | GUINESS      | 2006-02-15 04:34:33 |
|        2 | NICK        | WAHLBERG     | 2006-02-15 04:34:33 |
|        3 | ED          | CHASE        | 2006-02-15 04:34:33 |
|        4 | JENNIFER    | DAVIS        | 2006-02-15 04:34:33 |
|        5 | JOHNNY      | LOLLOBRIGIDA | 2006-02-15 04:34:33 |

In [ ]:
Logical Backups
==============================

-- does not involve copying of physical files.
-- all databases , some databases and table level backup can be taken.
-- utility have to be used like mysqldump amd mysqlpump.
-- takes backup in .sql format.

[mysqluser@mysqlserver backups]$ mysqldump --verbose --help

Dumping structure and contents of MySQL databases and tables.
Usage: mysqldump [OPTIONS] database [tables]
OR     mysqldump [OPTIONS] --databases [OPTIONS] DB1 [DB2 DB3...]
OR     mysqldump [OPTIONS] --all-databases [OPTIONS]


Single Table Backup
==================================

[mysqluser@mysqlserver backups]$ mysqldump sakila actor > sakilaDB_actorTable_bkp.sql


Whole Single Database Backup 
===================================

[mysqluser@mysqlserver backups]$ mysqldump sakila > sakilaDB_bkp.sql


Two databases backup
================================

[mysqluser@mysqlserver backups]$ mysqldump --databases posdb sakila > posdb_sakila_bkp.sql
[mysqluser@mysqlserver backups]$


All databases backup 
================================

[mysqluser@mysqlserver backups]$ mysqldump --all-databases > all_dbs_bkp.sql
[mysqluser@mysqlserver backups]$ 


Database backup with skipping some tables
=============================================

[mysqluser@mysqlserver backups]$ mysqldump sakila --ignore-table=sakila.actor --ignore-table=sakila.film_actor > 
sakiladb_skipactor_bkp.sql
[mysqluser@mysqlserver backups]$ 


Multiple database backup with skipping some tables 
=======================================================

[mysqluser@mysqlserver backups]$ mysqldump --databases sakila posdb --ignore-table=sakila.actor 
--ignore-table=sakila.film_actor --ignore-table=posdb.emp_salaries > sakiladb_posdb_ignoretables_bkp.sql
[mysqluser@mysqlserver backups]$ 



Single Table Restore from backup taken for single table
================================================================

[mysqluser@mysqlserver backups]$ cat sakilaDB_actorTable_bkp.sql | grep 'CREATE TABLE'

CREATE TABLE `actor`

[mysqluser@mysqlserver backups]$


mysql> drop table film_actor;
Query OK, 0 rows affected (0.28 sec)

mysql> drop table actor;
Query OK, 0 rows affected (0.01 sec)

mysql> select * from actor;
ERROR 1146 (42S02): Table 'sakila.actor' doesn't exist
mysql> 

[mysqluser@mysqlserver backups]$ mysql sakila < sakilaDB_actorTable_bkp.sql
[mysqluser@mysqlserver backups]$ 

mysql> select count(*) from actor;
+----------+
| count(*) |
+----------+
|      200 |
+----------+
1 row in set (0.13 sec)



Single Table restore from backup taken for full database
==========================================================

[mysqluser@mysqlserver backups]$ cat sakilaDB_bkp.sql | grep 'CREATE TABLE' | grep film_actor

CREATE TABLE `film_actor` 

[mysqluser@mysqlserver backups]$ 


ALL CREATE TABLE statements start with DROP TABLE IF EXISTS and then CREATE TABALE and then
INSERT STATEMENTS and it will end with keyword UNLOCK TABLES

[mysqluser@mysqlserver backups]$ sed -n -e '/DROP TABLE.*`film_actor`/,/UNLOCK TABLES/p' sakilaDB_bkp.sql 
> sakilaDB_filmactor_bkp.sql
[mysqluser@mysqlserver backups]$ 

[mysqluser@mysqlserver backups]$ mysql sakila < sakilaDB_filmactor_bkp.sql
[mysqluser@mysqlserver backups]$ 

mysql> select count(*) from film_actor;
+----------+
| count(*) |
+----------+
|     5462 |
+----------+
1 row in set (0.13 sec)


Full database restore from backup taken for a single database
=====================================================================

[mysqluser@mysqlserver backups]$ cat sakilaDB_bkp.sql | grep 'CREATE DATABASE'
[mysqluser@mysqlserver backups]$ 


mysql> drop database sakila;
Query OK, 23 rows affected (0.26 sec)

mysql> select count(*) from sakila.actor;
ERROR 1049 (42000): Unknown database 'sakila'
mysql> 
mysql> create database sakila;
Query OK, 1 row affected (0.19 sec)


[mysqluser@mysqlserver backups]$ mysql sakila < sakilaDB_bkp.sql
[mysqluser@mysqlserver backups]$ 

mysql> select count(*) from sakila.actor;
+----------+
| count(*) |
+----------+
|      200 |
+----------+
1 row in set (0.13 sec)



2 databases restore from backup taken for 2 databases 
============================================================

[mysqluser@mysqlserver backups]$ cat posdb_sakila_bkp.sql | grep 'CREATE DATABASE'

CREATE DATABASE /*!32312 IF NOT EXISTS*/ `posdb` /*!40100 DEFAULT CHARACTER SET utf8mb4 COLLATE utf8mb4_0900_ai_ci
 */ /*!80016 DEFAULT ENCRYPTION='N' */;
CREATE DATABASE /*!32312 IF NOT EXISTS*/ `sakila` /*!40100 DEFAULT CHARACTER SET utf8mb4 COLLATE utf8mb4_0900_ai_ci 
*/ /*!80016 DEFAULT ENCRYPTION='N' */;

[mysqluser@mysqlserver backups]$ 


mysql> drop database posdb;
Query OK, 2 rows affected (0.08 sec)

mysql> drop database sakila;
Query OK, 23 rows affected (0.23 sec)

[mysqluser@mysqlserver backups]$ mysql < posdb_sakila_bkp.sql
[mysqluser@mysqlserver backups]$ 

mysql> select count(*) from sakila.actor;
+----------+
| count(*) |
+----------+
|      200 |
+----------+

mysql> select count(*) from posdb.emp_details;
+----------+
| count(*) |
+----------+
|        3 |
+----------+


Restore all databases from backup taken for all databases
================================================================

[mysqluser@mysqlserver backups]$ cat all_dbs_bkp.sql | grep 'CREATE DATABASE' | grep posdb
CREATE DATABASE /*!32312 IF NOT EXISTS*/ `posdb` /*!40100 DEFAULT CHARACTER SET utf8mb4 COLLATE utf8mb4_0900_ai_ci 
*/ /*!80016 DEFAULT ENCRYPTION='N' */;
[mysqluser@mysqlserver backups]$


mysql> drop database information_schema;
ERROR 1044 (42000): Access denied for user 'root'@'localhost' to database 'information_schema'
mysql> 
mysql> drop database mysql;
ERROR 3552 (HY000): Access to system schema 'mysql' is rejected.
mysql> 
mysql> drop database performance_schema;
ERROR 1044 (42000): Access denied for user 'root'@'localhost' to database 'performance_schema'
mysql> 
mysql> drop database sys;
Query OK, 101 rows affected (0.20 sec)

mysql> drop database posdb;
Query OK, 2 rows affected (0.01 sec)

mysql> drop database sakila;
Query OK, 23 rows affected (0.23 sec)

mysql> show databases;
+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| performance_schema |
+--------------------+
3 rows in set (0.00 sec)


[mysqluser@mysqlserver backups]$ mysql < all_dbs_bkp.sql
[mysqluser@mysqlserver backups]$ 

mysql> show databases;
+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| performance_schema |
| posdb              |
| sakila             |
+--------------------+
5 rows in set (0.01 sec)


In [ ]:
Logical Backup - mysqlpump 
=======================================

-- is a newer version and is similar to mysqldump.
-- can use parallelism while backup.
-- compression can be used while backup.
-- user account backups can be taken.
-- shows progress of the backup.


MySQL utility for dumping data from databases to external file.
Usage: mysqlpump [OPTIONS] [--all-databases]
OR     mysqlpump [OPTIONS] --databases DB1 [DB2 DB3...]
OR     mysqlpump [OPTIONS] database [tables]


Use parallelism while taking backup 
============================================

[mysqluser@mysqlserver backups]$ mysqlpump --all-databases --default-parallelism=4 > all_dbs_bkp.sql
Dump progress: 1/1 tables, 0/0 rows
Dump progress: 24/26 tables, 678271/2429177 rows
Dump progress: 24/26 tables, 1455771/2429177 rows
Dump progress: 24/26 tables, 2198271/2429177 rows
Dump completed in 3472
[mysqluser@mysqlserver backups]$ 


[mysqluser@mysqlserver backups]$ mysqlpump --all-databases --default-parallelism=4 --parallel-schemas=3:ordersdb > 
all_dbs_bkp_2.sql
Dump progress: 1/1 tables, 0/0 rows
Dump progress: 24/26 tables, 681271/2429177 rows
Dump progress: 24/26 tables, 1400271/2429177 rows
Dump progress: 24/26 tables, 2092771/2429177 rows
Dump completed in 3865
[mysqluser@mysqlserver backups]$ 


Take user account backup 
============================================

[mysqluser@mysqlserver backups]$ mysqlpump --exclude-databases=% --users > user_account_bkp.sql
Dump completed in 144
[mysqluser@mysqlserver backups]$ 


Take compress backup with lZ4 or ZLIB
===============================================

[mysqluser@mysqlserver backups]$ mysqlpump --all-databases --compress-output=LZ4 > all_dbs_compressbkp.sql.lz4
Dump progress: 1/2 tables, 0/2 rows
Dump progress: 24/26 tables, 676271/2429177 rows
Dump progress: 24/26 tables, 1383021/2429177 rows
Dump progress: 24/26 tables, 2111271/2429177 rows
Dump completed in 3648
[mysqluser@mysqlserver backups]$ 

[mysqluser@mysqlserver backups]$ du -sh all_dbs_bkp.sql
129M	all_dbs_bkp.sql
[mysqluser@mysqlserver backups]$ du -sh all_dbs_compressbkp.sql.lz4
25M	all_dbs_compressbkp.sql.lz4
[mysqluser@mysqlserver backups]$ 

[mysqluser@mysqlserver backups]$ which lz4
/usr/bin/lz4
[mysqluser@mysqlserver backups]$ 
[mysqluser@mysqlserver backups]$ lz4 --help

[mysqluser@mysqlserver backups]$ lz4 -d all_dbs_compressbkp.sql.lz4 all_dbs_compressbkp.sql
all_dbs_compressbkp. : decoded 135102235 bytes                                 
[mysqluser@mysqlserver backups]$ 

In [ ]:
mysqldump or mysqlpump consistent backup 
====================================================

--single-transaction 
                      Creates a consistent snapshot by dumping all tables in a
                      single transaction. Works ONLY for tables stored in
                      storage engines which support multiversioning (currently
                      only InnoDB does); the dump is NOT guaranteed to be
                      consistent for other storage engines.

--skip-lock-tables 


[mysqluser@mysqlserver backups]$ mysqlpump --single-transaction > all_dbs_consistent_bkp.sql
Dump progress: 1/1 tables, 0/0 rows
Dump progress: 24/26 tables, 637521/2284901 rows
Dump progress: 24/26 tables, 1315521/2284901 rows
Dump progress: 24/26 tables, 1974271/2284901 rows
Dump completed in 4040
[mysqluser@mysqlserver backups]$ 

[mysqluser@mysqlserver backups]$ mysqldump --all-databases --single-transaction --skip-lock-tables > 
all_dbs_consistent_bkp_dump.sql
[mysqluser@mysqlserver backups]$ 


In [ ]:
MySql Hot Backup with Xtrabackup 
====================================================

-- Hot backup involves physically copying database related files to backup device
   when the database is Up.
-- tools - mysqlbackup ( oracle , paid) , mariadbbackup ( mariadb , open-source)
           xtrabackup ( percona , open-source )
-- point-in-time recovery can be performed using hot backup and binary logs.

In [ ]:
Install percona xtrabackup tool 
=================================================

https://www.percona.com/downloads#percona-xtrabackup

[root@mysqlserver Downloads]# yum localinstall percona-xtrabackup-80-8.0.30-23.1.el8.x86_64.rpm 
Last metadata expiration check: 2:33:22 ago on Tue 21 May 2024 10:43:33 AM IST.
Dependencies resolved.

Installed:
  libev-4.24-6.el8.x86_64                                 mariadb-connector-c-3.1.11-2.el8_3.x86_64                           
  percona-xtrabackup-80-8.0.30-23.1.el8.x86_64            perl-DBD-MySQL-4.046-3.module_el8.3.0+419+c2dec72b.x86_64           

Complete!
[root@mysqlserver Downloads]# 


[root@mysqlserver Downloads]# xtrabackup --version

xtrabackup version 8.0.30-23 based on MySQL server 8.0.30 Linux (x86_64) (revision id: 873b467185c)

[root@mysqlserver Downloads]# 


Directory for backups 
=====================================

[root@mysqlserver backups]# pwd
/home/mysqluser/backups
[root@mysqlserver backups]# 


create user for taking backups and grant permissions
====================================================================

mysql> create user 'bkpuser'@'%' identified by 'Bkpuser@2024';
Query OK, 0 rows affected (0.84 sec)

mysql> grant reload , lock tables , process , replication client on *.* to 'bkpuser'@'%' ;
Query OK, 0 rows affected (0.01 sec)

mysql> grant backup_admin on *.* to 'bkpuser'@'%' ;
Query OK, 0 rows affected (0.04 sec)

mysql> grant select on performance_schema.log_status to 'bkpuser'@'%' ;
Query OK, 0 rows affected (0.06 sec)

mysql> grant select on performance_schema.keyring_component_status to 'bkpuser'@'%' ;
Query OK, 0 rows affected (0.04 sec)

mysql> flush privileges;
Query OK, 0 rows affected (0.05 sec)


Setup credentials in mysql_config_editor for this user 
==============================================================

[root@mysqlserver backups]# mysql_config_editor set --login-path=bkp_user -u bkpuser -p
Enter password: 
[root@mysqlserver backups]# mysql_config_editor print --all
[client]
user = "root"
password = *****
[bkp_user]
user = "bkpuser"
password = *****
[root@mysqlserver backups]# 


Take full backup of mysql instance with xtrabackup , Also run a long transaction in the database
======================================================================================================

mysql> show tables;
+--------------------+
| Tables_in_ordersdb |
+--------------------+
| products01         |
| products02         |
+--------------------+

+------------+------------+
| Table      | Size in MB |
+------------+------------+
| products01 |       5.52 |
| products02 |      56.58 |
+------------+------------+

select max(prod_id) from ordersdb.products02;
'1100000'

mysql> call sp_insertprod_data (1100001);

mysql> select max(prod_id) from ordersdb.products02;
+--------------+
| max(prod_id) |
+--------------+
|      1238059 |
+--------------+


[root@mysqlserver backups]# xtrabackup --backup --target-dir=/home/mysqluser/backups

2024-05-21T13:48:17.880541+05:30 0 [Note] [MY-011825] [Xtrabackup] Transaction log of lsn (918626836) to (920516442)
 was copied.
2024-05-21T13:48:18.102870+05:30 0 [Note] [MY-011825] [Xtrabackup] completed OK!

[root@mysqlserver backups]# 


[root@mysqlserver backups]# xtrabackup --prepare --target-dir=/home/mysqluser/backups

2024-05-21T13:51:19.607355+05:30 0 [Note] [MY-012560] [InnoDB] The log sequence number 918571955 in the system 
tablespace does not match the log sequence number 918626836 in the redo log files!
2024-05-21T13:51:19.607369+05:30 0 [Note] [MY-012551] [InnoDB] Database was not shutdown normally!
2024-05-21T13:51:19.607376+05:30 0 [Note] [MY-012552] [InnoDB] Starting crash recovery.
2024-05-21T13:51:19.662145+05:30 0 [Note] [MY-013086] [InnoDB] Starting to parse redo log at lsn = 918626836, 
whereas checkpoint_lsn = 918626836 and start_lsn = 918626816
2024-05-21T13:51:19.711879+05:30 0 [Note] [MY-012550] [InnoDB] Doing recovery: scanned up to log sequence number 920502929
2024-05-21T13:51:19.750576+05:30 0 [Note] [MY-013083] [InnoDB] Log background threads are being started...
2024-05-21T13:51:20.221183+05:30 0 [Note] [MY-012532] [InnoDB] Applying a batch of 281 redo log records ...
2024-05-21T13:51:20.223172+05:30 0 [Note] [MY-012533] [InnoDB] 10%
2024-05-21T13:51:20.223246+05:30 0 [Note] [MY-012533] [InnoDB] 20%
2024-05-21T13:51:20.223278+05:30 0 [Note] [MY-012533] [InnoDB] 30%
2024-05-21T13:51:20.223289+05:30 0 [Note] [MY-012533] [InnoDB] 40%
2024-05-21T13:51:20.286509+05:30 0 [Note] [MY-012533] [InnoDB] 50%
2024-05-21T13:51:20.286656+05:30 0 [Note] [MY-012533] [InnoDB] 60%
2024-05-21T13:51:20.298030+05:30 0 [Note] [MY-012533] [InnoDB] 70%
2024-05-21T13:51:20.298092+05:30 0 [Note] [MY-012533] [InnoDB] 80%
2024-05-21T13:51:20.298106+05:30 0 [Note] [MY-012533] [InnoDB] 90%
2024-05-21T13:51:20.299966+05:30 0 [Note] [MY-012533] [InnoDB] 100%
2024-05-21T13:51:20.807306+05:30 0 [Note] [MY-012535] [InnoDB] Apply batch completed!
2024-05-21T13:51:20.992336+05:30 0 [Note] [MY-013084] [InnoDB] Log background threads are being closed...


2024-05-21T13:51:22.387957+05:30 0 [Note] [MY-011825] [Xtrabackup] starting shutdown with innodb_fast_shutdown = 1
2024-05-21T13:51:22.388274+05:30 0 [Note] [MY-012330] [InnoDB] FTS optimize thread exiting.
2024-05-21T13:51:23.322682+05:30 0 [Note] [MY-013072] [InnoDB] Starting shutdown...
2024-05-21T13:51:23.436743+05:30 0 [Note] [MY-013084] [InnoDB] Log background threads are being closed...
2024-05-21T13:51:23.470272+05:30 0 [Note] [MY-012980] [InnoDB] Shutdown completed; log sequence number 920503318
2024-05-21T13:51:23.472800+05:30 0 [Note] [MY-013251] [InnoDB] Number of pools: 1
2024-05-21T13:51:23.474048+05:30 0 [Note] [MY-011825] [Xtrabackup] completed OK!

[root@mysqlserver backups]# 


[root@mysqlserver backups]# pwd
/home/mysqluser/backups
[root@mysqlserver backups]# ls -rtl
total 141356
-rw-r-----. 1 root root 20971520 May 21 13:48  ibdata2
drwxr-x---. 2 root root       29 May 21 13:48  salesdb
drwxr-x---. 2 root root       50 May 21 13:48  ordersdb
-rw-r-----. 1 root root 31457280 May 21 13:48  mysql.ibd
drwxr-x---. 2 root root      143 May 21 13:48  mysql
drwxr-x---. 2 root root     8192 May 21 13:48  performance_schema
-rw-r-----. 1 root root      495 May 21 13:48  posb_binlog.000004
-rw-r-----. 1 root root       41 May 21 13:48  posb_binlog.index
-rw-r-----. 1 root root       23 May 21 13:48  xtrabackup_binlog_info
-rw-r-----. 1 root root     3960 May 21 13:48  ib_buffer_pool
-rw-r-----. 1 root root      479 May 21 13:48  backup-my.cnf
-rw-r-----. 1 root root      468 May 21 13:48  xtrabackup_info
-rw-r-----. 1 root root 16777216 May 21 13:51  undo_002
-rw-r-----. 1 root root 16777216 May 21 13:51  undo_001
-rw-r-----. 1 root root 12582912 May 21 13:51  ibtmp1
-rw-r-----. 1 root root 12582912 May 21 13:51  ibdata1
-rw-r-----. 1 root root 33554432 May 21 13:51  xtrabackup_logfile
-rw-r-----. 1 root root      145 May 21 13:51  xtrabackup_checkpoints
drwxr-x---. 2 root root        6 May 21 13:51 '#innodb_redo'
-rw-r-----. 1 root root       39 May 21 13:51  xtrabackup_tablespaces
[root@mysqlserver backups]# 



Notedown location of all files 
====================================================

[root@mysqlserver backups]# ls -rtl /var/lib/mysql
total 4
drwxr-xr-x. 2 mysql mysql   29 May 19 20:38 errorlog
drwxr-xr-x. 2 mysql mysql    6 May 19 20:45 tmpdir
drwxr-xr-x. 3 mysql mysql   26 May 20 17:54 redologs
drwxr-xr-x. 2 mysql mysql   81 May 20 17:56 doublewrite
drwxr-xr-x. 2 mysql mysql   38 May 20 17:56 undolog
drwxr-xr-x. 2 mysql mysql   86 May 21 13:04 systemdata
drwxr-xr-x. 2 mysql mysql  187 May 21 13:04 tempdata
drwxr-xr-x. 6 mysql mysql 4096 May 21 13:04 data
drwxr-xr-x. 2 mysql mysql  135 May 21 13:48 binlog


[root@mysqlserver backups]# ls -rtl /var/lib/mysql/errorlog/ >> files_location.txt
[root@mysqlserver backups]# vi files_location.txt 
[root@mysqlserver backups]# ls -rtl /var/lib/mysql/tmpdir/ >> files_location.txt
[root@mysqlserver backups]# vi files_location.txt 
[root@mysqlserver backups]# ls -rtl /var/lib/mysql/redologs/ >> files_location.txt
[root@mysqlserver backups]# vi files_location.txt 
[root@mysqlserver backups]# ls -rtl /var/lib/mysql/doublewrite/ >> files_location.txt
[root@mysqlserver backups]# vi files_location.txt 
[root@mysqlserver backups]# ls -rtl /var/lib/mysql/undolog/ >> files_location.txt
[root@mysqlserver backups]# vi files_location.txt 
[root@mysqlserver backups]# ls -rtl /var/lib/mysql/systemdata/ >> files_location.txt
[root@mysqlserver backups]# vi files_location.txt 
[root@mysqlserver backups]# ls -rtl /var/lib/mysql/tempdata/ >> files_location.txt
[root@mysqlserver backups]# vi files_location.txt 
[root@mysqlserver backups]# ls -rtl /var/lib/mysql/data/ >> files_location.txt
[root@mysqlserver backups]# vi files_location.txt 
[root@mysqlserver backups]# ls -rtl /var/lib/mysql/binlog/ >> files_location.txt
[root@mysqlserver backups]# 


Simulate data drive crash by removing some files 
=========================================================================

[root@mysqlserver data]# rm -rf mysql salesdb ordersdb
[root@mysqlserver data]# 

[root@mysqlserver systemdata]# rm -rf *
[root@mysqlserver systemdata]# 

[root@mysqlserver redologs]# rm -rf *
[root@mysqlserver redologs]# 


[root@mysqlserver systemdata]# systemctl restart mysqld
^C
[root@mysqlserver systemdata]# 
[root@mysqlserver systemdata]# systemctl stop mysqld
^C
[root@mysqlserver systemdata]# pidof mysqld
2014
[root@mysqlserver systemdata]# kill -9 2014
[root@mysqlserver systemdata]# 
[root@mysqlserver systemdata]# pidof mysqld
[root@mysqlserver systemdata]# 

[root@mysqlserver redologs]# tail -f /var/lib/mysql/errorlog/posb_mysqld.log
2024-05-21T09:38:23.286193Z 0 [ERROR] [MY-012894] [InnoDB] Unable to open '/var/lib/mysql/redologs/#innodb_redo/#ib_redo50'
 (error: 1504).
2024-05-21T09:38:23.286204Z 0 [ERROR] [MY-012894] [InnoDB] Unable to open '/var/lib/mysql/redologs/#innodb_redo/#ib_redo50' 
(error: 1504).
2024-05-21T09:38:23.286214Z 0 [ERROR] [MY-012894] [InnoDB] Unable to open '/var/lib/mysql/redologs/#innodb_redo/#ib_redo50' 
(error: 1504).


Restore using the backup taken through xtrabackup utility
=========================================================================


[root@mysqlserver backups]# xtrabackup --copy-back --target-dir=/home/mysqluser/backups/
xtrabackup version 8.0.30-23 based on MySQL server 8.0.30 Linux (x86_64) (revision id: 873b467185c)
2024-05-21T15:11:38.994383+05:30 0 [Note] [MY-011825] [Xtrabackup] cd to /home/mysqluser/backups/
2024-05-21T15:11:38.994880+05:30 0 [Note] [MY-011825] [Xtrabackup] Original data directory /var/lib/mysql/data is not empty!

[root@mysqlserver backups]# ls -rtl /var/lib/mysql/
total 0
drwxr-xr-x. 2 mysql mysql  29 May 19 20:38 errorlog
drwxr-xr-x. 2 mysql mysql   6 May 19 20:45 tmpdir
drwxr-xr-x. 2 mysql mysql  81 May 20 17:56 doublewrite
drwxr-xr-x. 2 mysql mysql 187 May 21 13:04 tempdata
drwxr-xr-x. 2 mysql mysql   6 May 21 15:05 redologs
drwxr-xr-x. 2 mysql mysql 239 May 21 15:06 binlog
drwxr-xr-x. 2 mysql mysql   6 May 21 15:13 data
drwxr-xr-x. 2 mysql mysql   6 May 21 15:14 systemdata
drwxr-xr-x. 2 mysql mysql   6 May 21 15:14 undolog
[root@mysqlserver backups]# 

[root@mysqlserver backups]# rm -rf /var/lib/mysql/tmpdir/*
[root@mysqlserver backups]# rm -rf /var/lib/mysql/doublewrite/*
[root@mysqlserver backups]# rm -rf /var/lib/mysql/tempdata/*
[root@mysqlserver backups]# rm -rf /var/lib/mysql/redologs/*
[root@mysqlserver backups]# rm -rf /var/lib/mysql/binlog/*
[root@mysqlserver backups]# rm -rf /var/lib/mysql/data/*
[root@mysqlserver backups]# rm -rf /var/lib/mysql/systemdata/*
[root@mysqlserver backups]# rm -rf /var/lib/mysql/undolog/*
[root@mysqlserver backups]# 


[root@mysqlserver backups]# xtrabackup --copy-back --target-dir=/home/mysqluser/backups/

xtrabackup version 8.0.30-23 based on MySQL server 8.0.30 Linux (x86_64) (revision id: 873b467185c)

2024-05-21T15:18:23.297326+05:30 0 [Note] [MY-011825] [Xtrabackup] cd to /home/mysqluser/backups/

2024-05-21T15:18:23.298020+05:30 0 [Note] [MY-011825] [Xtrabackup] Copying undo_001 to /var/lib/mysql/undolog/undo_001
2024-05-21T15:18:23.418784+05:30 0 [Note] [MY-011825] [Xtrabackup] Done: Copying undo_001 to /var/lib/mysql/undolog/undo_001
2024-05-21T15:18:23.441366+05:30 0 [Note] [MY-011825] [Xtrabackup] Copying undo_002 to /var/lib/mysql/undolog/undo_002
2024-05-21T15:18:23.607148+05:30 0 [Note] [MY-011825] [Xtrabackup] Done: Copying undo_002 to /var/lib/mysql/undolog/undo_00

2024-05-21T15:18:23.622115+05:30 0 [Note] [MY-011825] [Xtrabackup] Copying ibdata1 to /var/lib/mysql/systemdata/ibdata1
2024-05-21T15:18:23.741727+05:30 0 [Note] [MY-011825] [Xtrabackup] Done: Copying ibdata1 to /var/lib/mysql/systemdata/ibdata1
2024-05-21T15:18:23.762981+05:30 0 [Note] [MY-011825] [Xtrabackup] Copying ibdata2 to /var/lib/mysql/systemdata/ibdata2
2024-05-21T15:18:23.987501+05:30 0 [Note] [MY-011825] [Xtrabackup] Done: Copying ibdata2 to /var/lib/mysql/systemdata/ibdata2

2024-05-21T15:18:24.019295+05:30 0 [Note] [MY-011825] [Xtrabackup] Copying posb_binlog.000004 to /var/lib/mysql/binlog/posb_binlog.000004
2024-05-21T15:18:24.034744+05:30 0 [Note] [MY-011825] [Xtrabackup] Done: Copying posb_binlog.000004 to /var/lib/mysql/binlog/posb_binlog.000004
2024-05-21T15:18:24.053835+05:30 0 [Note] [MY-011825] [Xtrabackup] Copying posb_binlog.index to /var/lib/mysql/binlog/posb_binlog.index
2024-05-21T15:18:24.054001+05:30 0 [Note] [MY-011825] [Xtrabackup] Done: Copying posb_binlog.index to /var/lib/mysql/binlog/posb_binlog.index

2024-05-21T15:18:24.120472+05:30 1 [Note] [MY-011825] [Xtrabackup] Copying ./ordersdb/products01.ibd to /var/lib/mysql/data/ordersdb/products01.ibd
2024-05-21T15:18:24.254605+05:30 1 [Note] [MY-011825] [Xtrabackup] Done: Copying ./ordersdb/products01.ibd to /var/lib/mysql/data/ordersdb/products01.ibd
2024-05-21T15:18:24.262725+05:30 1 [Note] [MY-011825] [Xtrabackup] Copying ./ordersdb/products02.ibd to /var/lib/mysql/data/ordersdb/products02.ibd
2024-05-21T15:18:25.011519+05:30 1 [Note] [MY-011825] [Xtrabackup] Done: Copying ./ordersdb/products02.ibd to /var/lib/mysql/data/ordersdb/products02.ibd

2024-05-21T15:18:28.455806+05:30 1 [Note] [MY-011825] [Xtrabackup] Creating directory ./#innodb_redo
2024-05-21T15:18:28.455861+05:30 1 [Note] [MY-011825] [Xtrabackup] Done: creating directory ./#innodb_redo
2024-05-21T15:18:28.455948+05:30 1 [Note] [MY-011825] [Xtrabackup] Copying ./ibtmp1 to /var/lib/mysql/data/ibtmp1
2024-05-21T15:18:28.508881+05:30 1 [Note] [MY-011825] [Xtrabackup] Done: Copying ./ibtmp1 to /var/lib/mysql/data/ibtmp1
2024-05-21T15:18:28.546436+05:30 1 [Note] [MY-011825] [Xtrabackup] Copying ./files_location.txt to /var/lib/mysql/data/files_location.txt
2024-05-21T15:18:28.546578+05:30 1 [Note] [MY-011825] [Xtrabackup] Done: Copying ./files_location.txt to /var/lib/mysql/data/files_location.txt
2024-05-21T15:18:28.581307+05:30 0 [Note] [MY-011825] [Xtrabackup] completed OK!
[root@mysqlserver backups]# 


verify the copied files , change the owner to mysql  and start mysqld
=========================================================================

[root@mysqlserver backups]# chown -R mysql:mysql /var/lib/mysql/
[root@mysqlserver backups]# 

[root@mysqlserver backups]# systemctl start mysqld
[root@mysqlserver backups]# 
[root@mysqlserver backups]# systemctl status mysqld
● mysqld.service - MySQL Server
   Loaded: loaded (/usr/lib/systemd/system/mysqld.service; enabled; vendor preset: disabled)
   Active: active (running) since Tue 2024-05-21 15:44:17 IST; 41s ago


verify the databases and tables 
=======================================================

mysql> show databases;
+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| ordersdb           |
| performance_schema |
| salesdb            |
+--------------------+
5 rows in set (0.03 sec)


mysql> show databases;
+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| ordersdb           |
| performance_schema |
| salesdb            |
+--------------------+
5 rows in set (0.03 sec)

mysql> select max(prod_id) from ordersdb.products02;
+--------------+
| max(prod_id) |
+--------------+
|      1105690 |
+--------------+
1 row in set (0.12 sec)

We can see that all databases are restored , However the table ordersdb.products02 has lost the data
as the Transaction for this table was still running during the backup and we have nto done PITR.


In [ ]:
Point-In-Time Recovery using Xtrabackup and binary logs 
======================================================================

mysql> select count(*) from ordersdb.products01;
+----------+
| count(*) |
+----------+
|    46169 |
+----------+
1 row in set (0.18 sec)


run the full backup , at the same time run transaction on table ordersdb.products01
========================================================================================

mysql> mysql> call sp_insertprod_data(46170);


[root@mysqlserver backups]# xtrabackup --backup --target-dir=/home/mysqluser/backups

2024-05-21T17:52:47.879677+05:30 0 [Note] [MY-011825] [Xtrabackup] Writing /home/mysqluser/backups/backup-my.cnf
2024-05-21T17:52:47.879771+05:30 0 [Note] [MY-011825] [Xtrabackup] Done: Writing file /home/mysqluser/backups/backup-my.cnf
2024-05-21T17:52:47.892446+05:30 0 [Note] [MY-011825] [Xtrabackup] Writing /home/mysqluser/backups/xtrabackup_info
2024-05-21T17:52:47.892644+05:30 0 [Note] [MY-011825] [Xtrabackup] Done: Writing file /home/mysqluser/backups/xtrabackup_info
2024-05-21T17:52:48.916362+05:30 0 [Note] [MY-011825] [Xtrabackup] Transaction log of lsn (936726029) to (937399186) was copied.
2024-05-21T17:52:49.129399+05:30 0 [Note] [MY-011825] [Xtrabackup] completed OK!


[root@mysqlserver backups]# xtrabackup --prepare --target-dir=/home/mysqluser/backups

2024-05-21T17:53:47.269708+05:30 0 [Note] [MY-011825] [Xtrabackup] starting shutdown with innodb_fast_shutdown = 1
2024-05-21T17:53:47.269877+05:30 0 [Note] [MY-012330] [InnoDB] FTS optimize thread exiting.
2024-05-21T17:53:48.215344+05:30 0 [Note] [MY-013072] [InnoDB] Starting shutdown...
2024-05-21T17:53:48.352295+05:30 0 [Note] [MY-013084] [InnoDB] Log background threads are being closed...
2024-05-21T17:53:48.382924+05:30 0 [Note] [MY-012980] [InnoDB] Shutdown completed; log sequence number 937381910
2024-05-21T17:53:48.396703+05:30 0 [Note] [MY-013251] [InnoDB] Number of pools: 1
2024-05-21T17:53:48.397980+05:30 0 [Note] [MY-011825] [Xtrabackup] completed OK!


Note down the count of the table after the backup 
==========================================================

mysql> select count(*) from ordersdb.products01;
+----------+
| count(*) |
+----------+
|    72103 |
+----------+
1 row in set (0.16 sec)


simulate disk corruption OR accidental drop of the database 
======================================================================


mysql> drop database ordersdb;
Query OK, 2 rows affected (0.34 sec)

mysql> drop database salesdb;
Query OK, 1 row affected (0.09 sec)

mysql> select count(*) from ordersdb.products01;
ERROR 1049 (42000): Unknown database 'ordersdb'

mysql> show databases;
+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| performance_schema |
+--------------------+


prepare for restoration by emptying all the mysql data , systemdata , redolog directories 
==================================================================================================

Don't remove files in errorlog and binlog directories.

[root@mysqlserver backups]# systemctl stop mysqld
[root@mysqlserver backups]# 

[root@mysqlserver backups]# ls -rtl /var/lib/mysql/
total 4
drwxr-xr-x. 2 mysql mysql   29 May 19 20:38 errorlog
drwxr-xr-x. 2 mysql mysql    6 May 19 20:45 tmpdir
drwxr-xr-x. 2 mysql mysql   38 May 21 17:16 undolog
drwxr-xr-x. 3 mysql mysql   26 May 21 17:16 redologs
drwxr-xr-x. 2 mysql mysql   81 May 21 17:23 doublewrite
drwxr-xr-x. 2 mysql mysql  161 May 21 17:52 binlog
drwxr-xr-x. 4 mysql mysql 4096 May 21 18:05 data
drwxr-xr-x. 2 mysql mysql    6 May 21 18:05 tempdata
drwxr-xr-x. 2 mysql mysql   58 May 21 18:05 systemdata
 
[root@mysqlserver backups]# rm -rf /var/lib/mysql/tmpdir/*
[root@mysqlserver backups]# rm -rf /var/lib/mysql/undolog/*
[root@mysqlserver backups]# rm -rf /var/lib/mysql/redologs/*
[root@mysqlserver backups]# rm -rf /var/lib/mysql/doublewrite/*
[root@mysqlserver backups]# rm -rf /var/lib/mysql/data/*
[root@mysqlserver backups]# rm -rf /var/lib/mysql/tempdata/*
[root@mysqlserver backups]# rm -rf /var/lib/mysql/systemdata/*
[root@mysqlserver backups]# 


move binlogs to different location
===============================================================

[root@mysqlserver binlog]# mv posb_binlog.* ../old_binlog/
[root@mysqlserver binlog]# 


restore all the files from the full backup and verify
==============================================================


[root@mysqlserver backups]# xtrabackup --copy-back --target-dir=/home/mysqluser/backups

2024-05-21T18:15:44.800453+05:30 1 [Note] [MY-011825] [Xtrabackup] Creating directory ./#innodb_redo
2024-05-21T18:15:44.800524+05:30 1 [Note] [MY-011825] [Xtrabackup] Done: creating directory ./#innodb_redo
2024-05-21T18:15:44.800654+05:30 1 [Note] [MY-011825] [Xtrabackup] Copying ./ibtmp1 to /var/lib/mysql/data/ibtmp1
2024-05-21T18:15:44.828430+05:30 1 [Note] [MY-011825] [Xtrabackup] Done: Copying ./ibtmp1 to /var/lib/mysql/data/ibtmp1
2024-05-21T18:15:44.872181+05:30 0 [Note] [MY-011825] [Xtrabackup] completed OK!

[root@mysqlserver backups]# 

[root@mysqlserver backups]# ls -rtl /var/lib/mysql/data/
total 43028
drwxr-x---. 2 root root       29 May 21 18:15 salesdb
drwxr-x---. 2 root root       50 May 21 18:15 ordersdb
-rw-r-----. 1 root root 31457280 May 21 18:15 mysql.ibd
drwxr-x---. 2 root root      143 May 21 18:15 mysql
drwxr-x---. 2 root root     8192 May 21 18:15 performance_schema
-rw-r-----. 1 root root     3960 May 21 18:15 ib_buffer_pool
-rw-r-----. 1 root root      468 May 21 18:15 xtrabackup_info
-rw-r-----. 1 root root 12582912 May 21 18:15 ibtmp1
[root@mysqlserver backups]# 

[root@mysqlserver backups]# ls -rtl /var/lib/mysql/systemdata/
total 32768
-rw-r-----. 1 root root 12582912 May 21 18:15 ibdata1
-rw-r-----. 1 root root 20971520 May 21 18:15 ibdata2
[root@mysqlserver backups]# 

[root@mysqlserver backups]# chown -R mysql:mysql /var/lib/mysql/
[root@mysqlserver backups]# 



check the binary log files for the timing of database drop 
===============================================================================

[root@mysqlserver backups]# cat xtrabackup_binlog_info
posb_binlog.000010	493
[root@mysqlserver backups]# 

[root@mysqlserver old_binlog]# mysqlbinlog /var/lib/mysql/old_binlog/posb_binlog.000010 --start-position=493 > 
tempbinlog.sql
[root@mysqlserver old_binlog]# 

[root@mysqlserver old_binlog]# vi tempbinlog.sql
[root@mysqlserver old_binlog]# 

SET TIMESTAMP=1716294800/*!*/;
drop database ordersdb
/*!*/;
# at 8042510
#240521 18:03:30 server id 1  end_log_pos 8042587 CRC32 0xd5718456      
Anonymous_GTID  last_committed=23936    sequence_number=23937   rbr_only=no     
original_committed_timestamp=1716294810457076   immediate_commit_timestamp=1716294810457076     
transaction_length=190
# original_commit_timestamp=1716294810457076 (2024-05-21 18:03:30.457076 IST)
# immediate_commit_timestamp=1716294810457076 (2024-05-21 18:03:30.457076 IST)


create a restore file with this information 
=======================================================

[root@mysqlserver old_binlog]# mysqlbinlog /var/lib/mysql/old_binlog/posb_binlog.000010 --start-position=493 
--stop-datetime="2024-05-21 18:02:30" > restore_db.sql

[root@mysqlserver old_binlog]# 


start the mysqld server and run the restore_db.sql
=========================================================

[root@mysqlserver old_binlog]# systemctl start mysqld
[root@mysqlserver old_binlog]# 
[root@mysqlserver old_binlog]# systemctl status mysqld
● mysqld.service - MySQL Server
   Loaded: loaded (/usr/lib/systemd/system/mysqld.service; enabled; vendor preset: disabled)
   Active: active (running) since Tue 2024-05-21 18:29:48 IST; 22s ago
     Docs: man:mysqld(8)

[root@mysqlserver old_binlog]# mysql < restore_db.sql 
[root@mysqlserver old_binlog]# 


verify the database and table data
===================================================

mysql> show databases;
+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| ordersdb           |
| performance_schema |
| salesdb            |
+--------------------+

mysql> select count(*) from ordersdb.products01;
+----------+
| count(*) |
+----------+
|    72103 |
+----------+

In [ ]:
xtrabackup incremental backups 
=======================================


create a full base backup of the database
========================================================

mysql> select count(*) from ordersdb.products02;
+----------+
| count(*) |
+----------+
|        0 |
+----------+

mysql> call sp_insertprod_data(1);

mysql> select count(*) from ordersdb.products02;
+----------+
| count(*) |
+----------+
|    10000 |
+----------+

[root@mysqlserver backups]# xtrabackup --backup --target-dir=/home/mysqluser/backups/base

2024-05-21T21:10:06.158495+05:30 0 [Note] [MY-011825] [Xtrabackup] Writing /home/mysqluser/backups/base/xtrabackup_info
2024-05-21T21:10:06.158649+05:30 0 [Note] [MY-011825] [Xtrabackup] Done: Writing file /home/mysqluser/backups/base/xtrabackup_info
2024-05-21T21:10:07.163956+05:30 0 [Note] [MY-011825] [Xtrabackup] Transaction log of lsn (948614127) to (948614137) was copied.
2024-05-21T21:10:07.386730+05:30 0 [Note] [MY-011825] [Xtrabackup] completed OK!
[root@mysqlserver backups]# 

[root@mysqlserver backups]# ls -rtl
total 4
drwxr-x---. 6 root root 4096 May 21 21:10 base


create an incremental-1 database backup 
===========================================================

mysql> call sp_insertprod_data(10001);
Query OK, 1 row affected (6.03 sec)

mysql> select count(*) from ordersdb.products02;
+----------+
| count(*) |
+----------+
|    20000 |
+----------+
1 row in set (0.12 sec)

[root@mysqlserver backups]# xtrabackup --backup --target-dir=/home/mysqluser/backups/incr1 
--incremental-basedir=/home/mysqluser/backups/base

2024-05-21T21:14:27.111740+05:30 0 [Note] [MY-011825] [Xtrabackup] Writing /home/mysqluser/backups/incr1/backup-my.cnf
2024-05-21T21:14:27.111818+05:30 0 [Note] [MY-011825] [Xtrabackup] Done: Writing file 
/home/mysqluser/backups/incr1/backup-my.cnf
2024-05-21T21:14:27.112436+05:30 0 [Note] [MY-011825] [Xtrabackup] Writing /home/mysqluser/backups/incr1/xtrabackup_info
2024-05-21T21:14:27.112566+05:30 0 [Note] [MY-011825] [Xtrabackup] Done: Writing file 
/home/mysqluser/backups/incr1/xtrabackup_info
2024-05-21T21:14:28.114946+05:30 0 [Note] [MY-011825] [Xtrabackup] Transaction log of 
lsn (951863489) to (951863499) was copied.
2024-05-21T21:14:28.346979+05:30 0 [Note] [MY-011825] [Xtrabackup] completed OK!

[root@mysqlserver backups]# ls -rtl
total 8
drwxr-x---. 6 root root 4096 May 21 21:10 base
drwxr-x---. 6 root root 4096 May 21 21:14 incr1
[root@mysqlserver backups]# 



create an incremental-2 database backup 
===========================================================

mysql> call sp_insertprod_data(20001);
Query OK, 1 row affected (5.87 sec)

mysql> select count(*) from ordersdb.products02;
+----------+
| count(*) |
+----------+
|    30000 |
+----------+
1 row in set (0.11 sec)

[root@mysqlserver backups]# xtrabackup --backup --target-dir=/home/mysqluser/backups/incr2 
--incremental-basedir=/home/mysqluser/backups/incr1

2024-05-21T21:18:38.753260+05:30 0 [Note] [MY-011825] [Xtrabackup] Writing /home/mysqluser/backups/incr2/backup-my.cnf
2024-05-21T21:18:38.753330+05:30 0 [Note] [MY-011825] [Xtrabackup] Done: Writing file 
/home/mysqluser/backups/incr2/backup-my.cnf
2024-05-21T21:18:38.754091+05:30 0 [Note] [MY-011825] [Xtrabackup] Writing /home/mysqluser/backups/incr2/xtrabackup_info
2024-05-21T21:18:38.754256+05:30 0 [Note] [MY-011825] [Xtrabackup] Done: Writing file 
/home/mysqluser/backups/incr2/xtrabackup_info
2024-05-21T21:18:39.761943+05:30 0 [Note] [MY-011825] [Xtrabackup] Transaction log of
 lsn (955115596) to (955115606) was copied.
2024-05-21T21:18:39.999446+05:30 0 [Note] [MY-011825] [Xtrabackup] completed OK!


[root@mysqlserver backups]# ls -rtl
total 12
drwxr-x---. 6 root root 4096 May 21 21:10 base
drwxr-x---. 6 root root 4096 May 21 21:14 incr1
drwxr-x---. 6 root root 4096 May 21 21:18 incr2
[root@mysqlserver backups]# 


Insert more records 
=================================================

mysql> select count(*) from ordersdb.products02;
+----------+
| count(*) |
+----------+
|    80000 |
+----------+
1 row in set (0.12 sec)



Suppose a user accidently dropped the databases 
=====================================================


mysql> show databases;
+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| ordersdb           |
| performance_schema |
| salesdb            |
+--------------------+
5 rows in set (0.00 sec)

mysql> drop database ordersdb;
Query OK, 2 rows affected (0.06 sec)

mysql> drop database salesdb;
Query OK, 1 row affected (0.00 sec)

mysql> select count(*) from ordersdb.products02;
ERROR 1049 (42000): Unknown database 'ordersdb'
mysql> 
mysql> show databases;
+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| performance_schema |
+--------------------+
3 rows in set (0.00 sec)



prepare the backups for restoration
===================================================

Need to add --apply-log-only option when preparing incremental backups except for the last one.
This is to prevent the rollback phase , else the incremental backups becomes useless.


[root@mysqlserver backups]# xtrabackup --prepare --apply-log-only --target-dir=/home/mysqluser/backups/base

2024-05-21T21:31:03.970199+05:30 0 [Note] [MY-011825] [Xtrabackup] starting shutdown with innodb_fast_shutdown = 1
2024-05-21T21:31:03.970374+05:30 0 [Note] [MY-012330] [InnoDB] FTS optimize thread exiting.
2024-05-21T21:31:04.950115+05:30 0 [Note] [MY-013072] [InnoDB] Starting shutdown...
2024-05-21T21:31:05.055062+05:30 0 [Note] [MY-013084] [InnoDB] Log background threads are being closed...
2024-05-21T21:31:05.077729+05:30 0 [Note] [MY-012980] [InnoDB] Shutdown completed; log sequence number 948614137
2024-05-21T21:31:05.079997+05:30 0 [Note] [MY-013251] [InnoDB] Number of pools: 1
2024-05-21T21:31:05.081211+05:30 0 [Note] [MY-011825] [Xtrabackup] completed OK!


[root@mysqlserver backups]# xtrabackup --prepare --apply-log-only --target-dir=/home/mysqluser/backups/base 
--incremental-dir=/home/mysqluser/backups/incr1

2024-05-21T21:32:49.211777+05:30 0 [Note] [MY-011825] [Xtrabackup] Copying /home/mysqluser/backups/incr1/posb_binlog.000014 
to ./posb_binlog.000014
2024-05-21T21:32:49.215571+05:30 0 [Note] [MY-011825] [Xtrabackup] Done: 
Copying /home/mysqluser/backups/incr1/posb_binlog.000014 to ./posb_binlog.000014
2024-05-21T21:32:49.218088+05:30 0 [Note] [MY-011825] [Xtrabackup] Copying /home/mysqluser/backups/incr1/posb_binlog.index 
to ./posb_binlog.index
2024-05-21T21:32:49.218490+05:30 0 [Note] [MY-011825] [Xtrabackup] Done: 
Copying /home/mysqluser/backups/incr1/posb_binlog.index to ./posb_binlog.index
2024-05-21T21:32:49.220271+05:30 0 [Note] [MY-011825] [Xtrabackup] completed OK!


[root@mysqlserver backups]# xtrabackup --prepare --target-dir=/home/mysqluser/backups/base 
--incremental-dir=/home/mysqluser/backups/incr2

2024-05-21T21:34:53.240820+05:30 0 [Note] [MY-011825] [Xtrabackup] Done: 
Copying /home/mysqluser/backups/incr2//xtrabackup_tablespaces to ./xtrabackup_tablespaces
2024-05-21T21:34:53.241983+05:30 0 [Note] [MY-011825] [Xtrabackup] Copying /home/mysqluser/backups/incr2/posb_binlog.000015
 to ./posb_binlog.000015
2024-05-21T21:34:53.242126+05:30 0 [Note] [MY-011825] [Xtrabackup] 
Done: Copying /home/mysqluser/backups/incr2/posb_binlog.000015 to ./posb_binlog.000015
2024-05-21T21:34:53.243470+05:30 0 [Note] [MY-011825] [Xtrabackup] Copying /home/mysqluser/backups/incr2/posb_binlog.index 
to ./posb_binlog.index
2024-05-21T21:34:53.243624+05:30 0 [Note] [MY-011825] [Xtrabackup] 
Done: Copying /home/mysqluser/backups/incr2/posb_binlog.index to ./posb_binlog.index
2024-05-21T21:34:53.244710+05:30 0 [Note] [MY-011825] [Xtrabackup] completed OK!


stop the mysqld server and move the binary logs to another location
=================================================================================

moving binary logs is necessary due to conflict with xtrabackup copy.

[root@mysqlserver ~]# systemctl stop mysqld
[root@mysqlserver ~]# 

[root@mysqlserver binlog]# mv posb_binlog.* ../old_binlog/
[root@mysqlserver binlog]# 


remove all mysql data , system , redo , undo files 
==============================================================

don't remove files in errorlog and old_binlog directories.

[root@mysqlserver binlog]# ls -rtl /var/lib/mysql/
total 4
drwxr-xr-x. 2 mysql mysql   29 May 19 20:38 errorlog
drwxr-xr-x. 2 mysql mysql    6 May 19 20:45 tmpdir
drwxr-xr-x. 2 mysql mysql   38 May 21 18:15 undolog
drwxr-xr-x. 3 mysql mysql   26 May 21 18:15 redologs
drwxr-xr-x. 2 mysql mysql   81 May 21 18:29 doublewrite
drwxr-xr-x. 4 mysql mysql 4096 May 21 21:38 data
drwxr-xr-x. 2 mysql mysql    6 May 21 21:38 tempdata
drwxr-xr-x. 2 mysql mysql   58 May 21 21:38 systemdata
drwxr-xr-x. 2 mysql mysql  187 May 21 21:39 old_binlog
drwxr-xr-x. 2 mysql mysql    6 May 21 21:39 binlog
[root@mysqlserver binlog]# 

[root@mysqlserver binlog]# rm -rf /var/lib/mysql/tmpdir/*
[root@mysqlserver binlog]# rm -rf /var/lib/mysql/undolog/*
[root@mysqlserver binlog]# rm -rf /var/lib/mysql/redologs/*
[root@mysqlserver binlog]# rm -rf /var/lib/mysql/doublewrite/*
[root@mysqlserver binlog]# rm -rf /var/lib/mysql/data/*
[root@mysqlserver binlog]# rm -rf /var/lib/mysql/tempdata/*
[root@mysqlserver binlog]# rm -rf /var/lib/mysql/systemdata/*
[root@mysqlserver binlog]# rm -rf /var/lib/mysql/binlog/*
[root@mysqlserver binlog]# 


restore the files from the backup taken and verify 
=======================================================

[root@mysqlserver mysqluser]# xtrabackup --copy-back --target-dir=/home/mysqluser/backups/base

2024-05-21T21:49:02.206040+05:30 1 [Note] [MY-011825] [Xtrabackup] Creating directory ./#innodb_redo
2024-05-21T21:49:02.206087+05:30 1 [Note] [MY-011825] [Xtrabackup] Done: creating directory ./#innodb_redo
2024-05-21T21:49:02.206280+05:30 1 [Note] [MY-011825] [Xtrabackup] Copying ./ibtmp1 to /var/lib/mysql/data/ibtmp1
2024-05-21T21:49:02.240622+05:30 1 [Note] [MY-011825] [Xtrabackup] Done: Copying ./ibtmp1 to /var/lib/mysql/data/ibtmp1
2024-05-21T21:49:02.245845+05:30 1 [Note] [MY-011825] [Xtrabackup] Copying ./xtrabackup_info to /var/lib/mysql/data/xtrabackup_info
2024-05-21T21:49:02.245975+05:30 1 [Note] [MY-011825] [Xtrabackup] Done: Copying ./xtrabackup_info to /var/lib/mysql/data/xtrabackup_info
2024-05-21T21:49:02.281502+05:30 0 [Note] [MY-011825] [Xtrabackup] completed OK!
[root@mysqlserver mysqluser]# 

[root@mysqlserver mysqluser]# ls -rtl /var/lib/mysql/data/
total 43028
drwxr-x---. 2 root root       29 May 21 21:49 salesdb
drwxr-x---. 2 root root       50 May 21 21:49 ordersdb
-rw-r-----. 1 root root 31457280 May 21 21:49 mysql.ibd
drwxr-x---. 2 root root      143 May 21 21:49 mysql
drwxr-x---. 2 root root     8192 May 21 21:49 performance_schema
-rw-r-----. 1 root root     3960 May 21 21:49 ib_buffer_pool
-rw-r-----. 1 root root 12582912 May 21 21:49 ibtmp1
-rw-r-----. 1 root root      534 May 21 21:49 xtrabackup_info
[root@mysqlserver mysqluser]# 

[root@mysqlserver mysqluser]# ls -rtl /var/lib/mysql/systemdata/
total 32768
-rw-r-----. 1 root root 12582912 May 21 21:49 ibdata1
-rw-r-----. 1 root root 20971520 May 21 21:49 ibdata2
[root@mysqlserver mysqluser]# 
 
[root@mysqlserver mysqluser]# chown -R mysql:mysql /var/lib/mysql/
[root@mysqlserver mysqluser]# 


prepare the binlogs to apply after the incremental backup 
=====================================================================

[root@mysqlserver base]# cat xtrabackup_binlog_info
posb_binlog.000015	157
[root@mysqlserver base]# 

[root@mysqlserver old_binlog]# mysqlbinlog /var/lib/mysql/old_binlog/posb_binlog.000015 --start-position=157
 > checkbinlog.sql
[root@mysqlserver old_binlog]# 

[root@mysqlserver old_binlog]# vi checkbinlog.sql 
[root@mysqlserver old_binlog]#

drop database ordersdb
/*!*/;
# at 16800350
#240521 21:25:18 server id 1  end_log_pos 16800427 CRC32 0x7006a5f2    
Anonymous_GTID  last_committed=50001    sequence_number=50002   rbr_only=no     
original_committed_timestamp=1716306918034889   immediate_commit_timestamp=1716306918034889     
transaction_length=190
# original_commit_timestamp=1716306918034889 (2024-05-21 21:25:18.034889 IST)
# immediate_commit_timestamp=1716306918034889 (2024-05-21 21:25:18.034889 IST)

[root@mysqlserver old_binlog]# mysqlbinlog /var/lib/mysql/old_binlog/posb_binlog.000015 --start-position=157 
--stop-datetime="2024-05-21 21:24:00" > restorelog.sql
[root@mysqlserver old_binlog]# 


start the mysqld database server and restore the restorelog file 
==============================================================================

[root@mysqlserver old_binlog]# systemctl start mysqld
[root@mysqlserver old_binlog]# 
[root@mysqlserver old_binlog]# systemctl status mysqld
● mysqld.service - MySQL Server
   Loaded: loaded (/usr/lib/systemd/system/mysqld.service; enabled; vendor preset: disabled)
   Active: active (running) since Tue 2024-05-21 22:03:09 IST; 9s ago
     Docs: man:mysqld(8)

[root@mysqlserver old_binlog]# mysql < restorelog.sql
[root@mysqlserver old_binlog]# 


mysql> show databases;
+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| ordersdb           |
| performance_schema |
| salesdb            |
+--------------------+
5 rows in set (0.01 sec)

mysql> select count(*) from ordersdb.products02;
+----------+
| count(*) |
+----------+
|    80000 |
+----------+
1 row in set (0.15 sec)

In [ ]:
xtrabackup compressed backup 
================================================

compressed backup can be taken by adding --compress option
=====================================================================

[root@mysqlserver backups]# xtrabackup --backup --compress --target-dir=/home/mysqluser/backups/full_compressed

2024-05-22T00:34:26.784221+05:30 2 [Note] [MY-011825] [Xtrabackup] 
Done: Compressing /var/lib/mysql/systemdata/ibdata2 to /home/mysqluser/backups/full_compressed/ibdata2.qp
2024-05-22T00:34:26.804039+05:30 2 [Note] [MY-011825] [Xtrabackup] 
Compressing ./salesdb/products101.ibd to /home/mysqluser/backups/full_compressed/salesdb/products101.ibd.qp
2024-05-22T00:34:27.142967+05:30 2 [Note] [MY-011825] [Xtrabackup] 
Done: Compressing ./salesdb/products101.ibd to /home/mysqluser/backups/full_compressed/salesdb/products101.ibd.qp
2024-05-22T00:34:27.157905+05:30 2 [Note] [MY-011825] [Xtrabackup] 
Compressing ./ordersdb/products01.ibd to /home/mysqluser/backups/full_compressed/ordersdb/products01.ibd.qp

2024-05-22T00:34:29.442917+05:30 0 [Note] [MY-011825] [Xtrabackup] MySQL binlog position: 
filename 'posb_binlog.000019', position '157'
2024-05-22T00:34:29.443240+05:30 0 [Note] [MY-011825] [Xtrabackup] 
Compressing /home/mysqluser/backups/full_compressed/backup-my.cnf.qp
2024-05-22T00:34:29.443463+05:30 0 [Note] [MY-011825] [Xtrabackup] 
Done: Compressing file /home/mysqluser/backups/full_compressed/backup-my.cnf.qp
2024-05-22T00:34:29.445863+05:30 0 [Note] [MY-011825] [Xtrabackup] 
Compressing /home/mysqluser/backups/full_compressed/xtrabackup_info.qp
2024-05-22T00:34:29.446132+05:30 0 [Note] [MY-011825] [Xtrabackup] 
Done: Compressing file /home/mysqluser/backups/full_compressed/xtrabackup_info.qp
2024-05-22T00:34:30.448837+05:30 0 [Note] [MY-011825] [Xtrabackup] 
Transaction log of lsn (971505936) to (971505946) was copied.
2024-05-22T00:34:30.671305+05:30 0 [Note] [MY-011825] [Xtrabackup] completed OK!

[root@mysqlserver backups]# du -sh full
188M	full

[root@mysqlserver backups]# du -sh full_compressed
28M	full_compressed



compressed backups should be decompressed before prepare and restore
=============================================================================

[root@mysqlserver backups]# xtrabackup --decompress --target-dir=/home/mysqluser/backups/full_compressed

2024-05-22T00:41:11.573143+05:30 0 [Note] [MY-011825] [Xtrabackup] decompressing ./xtrabackup_logfile.qp
sh: qpress: command not found
2024-05-22T00:41:11.673498+05:30 0 [ERROR] [MY-011825] [Xtrabackup] decrypt and decompress thread 0 failed.

[root@mysqlserver Downloads]# yum localinstall qpress-20220819-3.el8.x86_64.rpm

Installed:
  qpress-20220819-3.el8.x86_64                                                                                         

Complete!
[root@mysqlserver Downloads]# 

[root@mysqlserver Downloads]# which qpress
/usr/bin/qpress
[root@mysqlserver Downloads]# 

[root@mysqlserver backups]# xtrabackup --decompress --target-dir=/home/mysqluser/backups/full_compressed

2024-05-22T00:51:55.703139+05:30 0 [Note] [MY-011825] [Xtrabackup] decompressing ./posb_binlog.000019.qp
2024-05-22T00:51:55.744925+05:30 0 [Note] [MY-011825] [Xtrabackup] decompressing ./posb_binlog.index.qp
2024-05-22T00:51:55.753245+05:30 0 [Note] [MY-011825] [Xtrabackup] decompressing ./xtrabackup_binlog_info.qp
2024-05-22T00:51:55.791007+05:30 0 [Note] [MY-011825] [Xtrabackup] decompressing ./ib_buffer_pool.qp
2024-05-22T00:51:55.838389+05:30 0 [Note] [MY-011825] [Xtrabackup] decompressing ./backup-my.cnf.qp
2024-05-22T00:51:55.844887+05:30 0 [Note] [MY-011825] [Xtrabackup] decompressing ./xtrabackup_info.qp
2024-05-22T00:51:55.885948+05:30 0 [Note] [MY-011825] [Xtrabackup] decompressing ./xtrabackup_tablespaces.qp
2024-05-22T00:51:55.980879+05:30 0 [Note] [MY-011825] [Xtrabackup] completed OK!
[root@mysqlserver backups]# 

[root@mysqlserver backups]# du -sh full_compressed/
215M	full_compressed/
[root@mysqlserver backups]# du -sh full/
188M	full/
[root@mysqlserver backups]# 

In [ ]:
Creating partial backups
============================================


all tables in a database 
=================================

[root]# xtrabackup --backup --target-dir=/home/mysqluser/backups/ordersdb_all_tables/ --tables="^ordersdb[.].*"

2024-05-22T01:08:17.742579+05:30 0 [Note] [MY-011825] [Xtrabackup] Done: 
Writing file /home/mysqluser/backups/ordersdb_all_tables/xtrabackup_info
2024-05-22T01:08:18.752466+05:30 0 [Note] [MY-011825] [Xtrabackup] 
Transaction log of lsn (971505956) to (971505966) was copied.
2024-05-22T01:08:19.001043+05:30 0 [Note] [MY-011825] [Xtrabackup] completed OK!
[root@mysqlserver backups]# 

[root@mysqlserver ordersdb_all_tables]# pwd
/home/mysqluser/backups/ordersdb_all_tables

[root@mysqlserver ordersdb_all_tables]# ls -rtl
total 96292
-rw-r-----. 1 root root 12582912 May 22 01:08 ibdata1
-rw-r-----. 1 root root 20971520 May 22 01:08 ibdata2
drwxr-x---. 2 root root       50 May 22 01:08 ordersdb
-rw-r-----. 1 root root 31457280 May 22 01:08 mysql.ibd
-rw-r-----. 1 root root 16777216 May 22 01:08 undo_002
-rw-r-----. 1 root root 16777216 May 22 01:08 undo_001
-rw-r-----. 1 root root      157 May 22 01:08 posb_binlog.000020
-rw-r-----. 1 root root       41 May 22 01:08 posb_binlog.index
-rw-r-----. 1 root root       23 May 22 01:08 xtrabackup_binlog_info
-rw-r-----. 1 root root     2560 May 22 01:08 xtrabackup_logfile
-rw-r-----. 1 root root      139 May 22 01:08 xtrabackup_checkpoints
-rw-r-----. 1 root root     3960 May 22 01:08 ib_buffer_pool
-rw-r-----. 1 root root      479 May 22 01:08 backup-my.cnf
-rw-r-----. 1 root root      513 May 22 01:08 xtrabackup_info
-rw-r-----. 1 root root       39 May 22 01:08 xtrabackup_tablespaces
[root@mysqlserver ordersdb_all_tables]# 


single table in a database 
====================================

[root@mysqlserver backups]# xtrabackup --backup --target-dir=/home/mysqluser/backups/ordersdb_products01_table/ 
--tables="^ordersdb[.]products01"


table files option 
================================

[root@mysqlserver backups]# cat ordersdb_tables.txt
ordersdb.products01
ordersdb.products02

[root@mysqlserver backups]# xtrabackup --backup --target-dir=/home/mysqluser/backups/ordersdb_tables/ 
--tables-file=/home/mysqluser/backups/ordersdb_tables.txt


single database backup
=========================================

[root@mysqlserver backups]# xtrabackup --backup --databases='mysql sys performance_schema ordersdb' 
--target-dir=/home/mysqluser/backups/ordersdb_bkp/


database files option
===========================================

[root@mysqlserver backups]# cat databases_for_bkp.txt 
mysql
sys
performance_schema
salesdb
[root@mysqlserver backups]# 


[root@mysqlserver backups]# xtrabackup --backup --databases-file=/home/mysqluser/backups/databases_for_bkp.txt
 --target-dir=/home/mysqluser/backups/databases_bkp/


In [ ]:
preparing partial backups and restore 
==========================================

[root@mysqlserver backups]# xtrabackup --prepare --export --target-dir=/home/mysqluser/backups/ordersdb_all_tables


suppose one of the table products01 is truncated.
==========================================================


mysql> select count(*) from ordersdb.products01;
+----------+
| count(*) |
+----------+
|    72103 |
+----------+

mysql> truncate table ordersdb.products01;

mysql> select count(*) from ordersdb.products01;
+----------+
| count(*) |
+----------+
|        0 |
+----------+

mysql> alter table ordersdb.products01 discard tablespace;
Query OK, 0 rows affected (0.00 sec)

[root@mysqlserver ordersdb]# cp /home/mysqluser/backups/ordersdb_all_tables/ordersdb/products01.* 
/var/lib/mysql/data/ordersdb/
[root@mysqlserver ordersdb]# chown mysql:mysql /var/lib/mysql/data/ordersdb/products01.*
[root@mysqlserver ordersdb]# 


mysql> alter table ordersdb.products01 import tablespace;
Query OK, 0 rows affected (0.13 sec)

mysql> select count(*) from ordersdb.products01;
+----------+
| count(*) |
+----------+
|    72103 |
+----------+
1 row in set (0.13 sec)


suppose one of the table products02 is dropped
==========================================================

mysql> drop table products02;
Query OK, 0 rows affected (0.25 sec)

CREATE TABLE `products02` (
  `prod_id` int NOT NULL,
  `prod_details` varchar(100) DEFAULT NULL,
  `prod_date` datetime DEFAULT NULL,
  PRIMARY KEY (`prod_id`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci;

alter table products02 discard tablespace;

[root@mysqlserver ordersdb]# cp /home/mysqluser/backups/ordersdb_all_tables/ordersdb/products02.* 
/var/lib/mysql/data/ordersdb/
[root@mysqlserver ordersdb]# 
[root@mysqlserver ordersdb]# chown mysql:mysql /var/lib/mysql/data/ordersdb/products02.*
[root@mysqlserver ordersdb]# 

alter table products02 import tablespace;

mysql> select count(*) from ordersdb.products02;
+----------+
| count(*) |
+----------+
|    80000 |
+----------+
1 row in set (0.11 sec)